## GeoCat experimental

In [1]:
import numpy as np
import pandas as pd
import recommenders as rec
import geocat
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math
import timeit
from IPython.core.debugger import set_trace
%matplotlib inline
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)

CITY="madison"


In [2]:
df_checkin=pd.read_csv("../data/checkin/"+CITY+".csv",converters={'categories':string_to_array})
df_checkin['date']=pd.to_datetime(df_checkin['date'])
# users=df_checkin['user_id'].drop_duplicates().reset_index(drop=True)
# num_users=len(users)
df_checkin.set_index("user_id",inplace=True)
df_checkin.head()

,business_id,date,latitude,longitude,categories,subarea_id
user_id,,,,,,
MB7Z80PJxNyQM7keDpR2XQ,br1uQbc3YU5jCpYi46nIgQ,2018-04-04 00:09:03,43.069425,-89.389567,"[Coffee & Tea, Cafes]",2064.0
4HwS2oCVQ1-f_kzHCv5X3A,br1uQbc3YU5jCpYi46nIgQ,2018-02-16 20:40:41,43.069425,-89.389567,"[Coffee & Tea, Cafes]",2064.0
OLBH2Ds1wllrzpHlyojzJQ,br1uQbc3YU5jCpYi46nIgQ,2018-04-12 11:31:03,43.069425,-89.389567,"[Coffee & Tea, Cafes]",2064.0
wd3xoNaDLib8dhQ7BxUl6g,br1uQbc3YU5jCpYi46nIgQ,2018-03-07 14:08:41,43.069425,-89.389567,"[Coffee & Tea, Cafes]",2064.0
4HwS2oCVQ1-f_kzHCv5X3A,br1uQbc3YU5jCpYi46nIgQ,2018-02-16 20:48:49,43.069425,-89.389567,"[Coffee & Tea, Cafes]",2064.0


In [3]:
df_poi=pd.read_csv("../data/poi/"+CITY+".csv",converters={'categories':string_to_array})
df_poi=df_poi.set_index("business_id")

# Test and training set

In [4]:
df_checkin=df_checkin.sort_values(['user_id','date'])

ranks=df_checkin.groupby('user_id')['date'].rank(method='first').reset_index(drop=True)
ranks.head()

#ranks=df_review.groupby('user_id')['date'].rank(method='first')
count=df_checkin.reset_index()['user_id'].map(df_checkin.groupby('user_id')['date'].apply(len))
count.head()

training_set_delimiter=(ranks/count)<0.72
test_set_delimiter=np.logical_not(training_set_delimiter)
training_set_delimiter

df_checkin_train=df_checkin.reset_index()[training_set_delimiter]
df_checkin_train.count()

df_checkin_test=df_checkin.reset_index()[test_set_delimiter]
df_checkin_test.count()

users=df_checkin_train['user_id'].drop_duplicates().reset_index(drop=True)



In [5]:
dict_alias_title,category_tree,dict_alias_depth=geocat.category_manipulation_utils()
undirected_category_tree=category_tree.to_undirected()

In [6]:
num_users=len(users)
users.count()

1022

In [7]:
#df_checkin_train[['user_id','business_id','date']].groupby(['user_id','business_id']).count().reset_index().to_csv("../data/checkin/train/"+CITY+".csv",index=False,header=False)

#df_checkin_test[['user_id','business_id','date']].groupby(['user_id','business_id']).count().reset_index().to_csv("../data/checkin/test/"+CITY+".csv",index=False,header=False)

In [8]:
df_checkin_train=df_checkin_train.set_index('user_id')
df_checkin_test=df_checkin_test.set_index('user_id')

### load poi neighbor

In [9]:
df_poi_neighbor=pd.read_csv("../data/neighbor/"+CITY+".csv",converters={'neighbors':string_to_array})
df_poi_neighbor=df_poi_neighbor.set_index("business_id")

In [10]:
df_checkin_train.loc["scaxsfOynKUZEijn_UCTkg"].count()


business_id    33
date           33
latitude       33
longitude      33
categories     33
subarea_id     33
dtype: int64

In [16]:
user_id="scaxsfOynKUZEijn_UCTkg"
df_checkin.loc[[user_id]]


,business_id,date,latitude,longitude,categories,subarea_id
user_id,,,,,,
scaxsfOynKUZEijn_UCTkg,OSXb-DOITpDR7nm8wjrZ2w,2017-02-28 00:05:42,43.071577,-89.524585,[Nail Salons],2086.0
scaxsfOynKUZEijn_UCTkg,U7G4VBrra439LgGBvV7dcg,2017-02-28 16:23:37,43.123634,-89.350392,[Eyelash Service],2897.0
scaxsfOynKUZEijn_UCTkg,_q0ci_oTC-QccjFKVc6CeA,2017-03-02 05:55:47,43.095776,-89.511771,"[Cupcakes, Custom Cakes, Bakeries]",2503.0
scaxsfOynKUZEijn_UCTkg,qAzz1OYoGjwcSdwzu6FgTg,2017-03-03 05:53:17,43.092666,-89.526329,[Nail Salons],2436.0
scaxsfOynKUZEijn_UCTkg,Iq1P8S3--gqd50xoQJxpqA,2017-03-03 05:59:59,43.072930,-89.453214,"[Juice Bars & Smoothies, Specialty Food, Fruit...",2144.0
scaxsfOynKUZEijn_UCTkg,kQ81FVLQWKa9xEqZGQV_ng,2017-03-05 05:48:50,43.090543,-89.524149,[Coffee & Tea],2435.0
scaxsfOynKUZEijn_UCTkg,hySu4oYN7R3sSohuSeEsTg,2017-03-05 16:16:49,43.071577,-89.524585,"[Cycling Classes, Gyms, Fitness & Instruction]",2086.0
scaxsfOynKUZEijn_UCTkg,VfWX3UCKvUnVktdOuR8TwA,2017-03-05 23:37:52,43.071461,-89.452308,"[Dim Sum, Chinese, Vegan, Vegetarian]",2074.0
scaxsfOynKUZEijn_UCTkg,eZ1u1mwjUqb62v9dSiqZaQ,2017-03-10 00:32:38,43.057013,-89.497325,"[Fashion, Uniforms, Women's Clothing, Shoe Sto...",1871.0


In [12]:
N=80
K=20
VERY_SMALL_VALUE=-100
i=0

# tmp_rec_list=rec.mostpopular(df_checkin,K)
# tmp_rec_list['score']=tmp_rec_list['score']/tmp_rec_list['score'].max()
# print(tmp_rec_list)
dict_rec_list=dict()
for user_id in users:
    user_id="scaxsfOynKUZEijn_UCTkg"
    print(user_id)
    # User visits
    df_user_checkin=df_checkin_train.loc[[user_id]].reset_index(drop=True)
    
    list_user_pois=set(df_user_checkin['business_id'].drop_duplicates().values)
    
    
    tmp_rec_list=rec.mostpopular(df_checkin_train.reset_index(),N,num_users,list_user_pois)
    
    # add other columns
    tmp_rec_list=pd.merge(tmp_rec_list,df_poi.reset_index(),on='business_id').drop_duplicates(subset=df_poi.reset_index().columns.difference(['categories'])).reset_index(drop=True)

    business_cover=dict()
    for business_id in df_user_checkin['business_id'].drop_duplicates().reset_index(drop=True):
        business_cover[business_id]=0
    # Constants
    #COVER_OF_POI=len(df_user_review)/len(tmp_rec_list)
    div_geo_cat_weight = 0.5 # beta,this is here because of the work to be done on parameter customization for each user
    div_weight = 0.75 # lambda, geo vs cat
    current_proportionality=0
    
    rec_list=pd.DataFrame(columns=['business_id','score','latitude','longitude','categories'])
    relevant_categories_user=geocat.relevant_categories_to_the_user(df_user_checkin)
    
    # post-processing for diversification
    for i in range(K):
#         start = timeit.default_timer()
        #num_current_candidates=len(tmp_rec_list)
        poi_to_insert=None
        max_objective_value=VERY_SMALL_VALUE
        rec_list_categories=geocat.category_in_rec_list(tmp_rec_list)
        
        for index,poi_candidate in tmp_rec_list.iterrows():
            poi_neighbors=df_poi.loc[df_poi_neighbor.loc[poi_candidate.business_id].neighbors].loc[
                set(df_poi_neighbor.loc[poi_candidate.business_id].neighbors) & list_user_pois]
            print(poi_neighbors)

            objective_value=geocat.objective_ILD_GC_PR(poi_candidate,df_user_checkin,rec_list,K,\
                                                business_cover,current_proportionality,\
                                               div_geo_cat_weight,div_weight,dict_alias_title,undirected_category_tree,\
                                                      relevant_categories_user,rec_list_categories,poi_neighbors)
            
            if objective_value > max_objective_value:
                max_objective_value=objective_value
                poi_to_insert=poi_candidate 
#         stop = timeit.default_timer()
#         print('Time:', stop - start)
        if poi_to_insert is not None:
            poi_neighbors=df_poi_neighbor.loc[poi_to_insert.business_id].neighbors
            poi_to_insert['score']=max_objective_value
            rec_list=rec_list.append(poi_to_insert)
            # remove from tmp_rec_list
            tmp_rec_list=tmp_rec_list[tmp_rec_list['business_id']!=poi_to_insert['business_id']].reset_index(drop=True)
            
            current_proportionality=geocat.update_geo_cov(poi_to_insert,df_user_checkin,K,business_cover,poi_neighbors)
    
    dict_rec_list[user_id]=rec_list[['business_id','score']]
    print(rec_list[['business_id','score']])
    break
    i+=1
    if i % num_users==0:
        print(i)

scaxsfOynKUZEijn_UCTkg
> /home/heitor/recsys/algorithms/geocat.py(92)relevant_categories_to_the_user()
     90 def relevant_categories_to_the_user(df_user_checkin):
     91     set_trace()
---> 92     df_num_checkins=df_user_checkin.groupby("business_id").count()['date']
     93 
     94     mean_poi_visits=df_num_checkins.mean()

ipdb> q


BdbQuit: 

0   IMLrj2klosTFvPRLv56cng  0.260358
24  ujmEBvifdJM6h6RLv4wQIg  0.050891
68  V13IWgZvNps2Vo7WyEnD8A  0.024049
0   XXW_OFaYQkkGOGniujZFHg  0.000000
0   FvVSy2r7_zDEhZWqLgjXNQ  0.000000
0   WRQ-9LluyivReFiQZFUujw  0.000000
0   Vs7gc9EE3k9wARuUcN9piA  0.000000
0   Os1n1_idfw9vv9kwULGJnQ  0.000000
0   qsJkjQoM3Hdg6Gtqp2S9_w  0.000000
0   O7UMzd3i-Zk8dMeyY9ZwoA  0.000000
0   qmymSqVwHYRqdwfcBatzpQ  0.000000
0   pH0BLkL4cbxKzu471VZnuA  0.000000
0   78hGCrQ6wLA2yh8zMywnmw  0.000000
0   _pBXtjN43eqMV0XZTz7nmw  0.000000
0   tMYYSWyTppDcF070Hmy-kQ  0.000000
0   dm6sO_Y8JdKTE1ZM955yug  0.000000
0   Yiw2jUJzUdFmglHZgj4RaQ  0.000000
0   FhIeCF6QrsLaRvAeu0oEPQ  0.000000
0   5ma1OUQWy_Ds80xUiP7JPQ  0.000000
0   k2b3niokS_tosjah_rzCPw  0.000000